<a href="https://githubtocolab.com/gee-community/geemap/blob/master/docs/notebooks/114_dynamic_world.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

**Creating near real-time global 10-m land cover maps with geemap and Dynamic World**

- App: <https://www.dynamicworld.app>
- App2: <https://earthoutreach.users.earthengine.app/view/dynamicworld>
- Paper: <https://doi.org/10.1038/s41597-022-01307-4>
- Model: <https://github.com/google/dynamicworld>
- Training data: <https://doi.pangaea.de/10.1594/PANGAEA.933475>
- Data: <https://developers.google.com/earth-engine/datasets/catalog/GOOGLE_DYNAMICWORLD_V1>
- JavaScript tutorial: <https://developers.google.com/earth-engine/tutorials/community/introduction-to-dynamic-world-pt-1>

In [1]:
import ee
import geemap

In [4]:
ee.Authenticate()
ee.Initialize(project='nodal-talon-453619-p2')
Map = geemap.Map(ee_initialize=False)
Map.add_basemap("HYBRID")
Map

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…

In [21]:
# Set the region of interest by simply drawing a polygon on the map
region = Map.user_roi

# Or, set your custom shapefile asset path here (e.g., "users/your_username/your_shapefile")
asset_roi_path = "projects/nodal-talon-453619-p2/assets/Bengaluru_District" # Change this to your asset path, e.g., "projects/nodal-talon-453619-p2/assets/my_shapefile"

if region is None:
    if asset_roi_path:
        region = ee.FeatureCollection(asset_roi_path)
    else:
        # Default to Bengaluru District if no user drawing or custom asset is provided
        region = ee.FeatureCollection("projects/nodal-talon-453619-p2/assets/Bengaluru_District")

Map.centerObject(region)

In [23]:
# Set the date range
start_date = "2015-01-01"
end_date = "2016-01-01"

In [24]:
# Create a Sentinel-2 image composite
image = geemap.dynamic_world_s2(region, start_date, end_date)
vis_params = {"bands": ["B4", "B3", "B2"], "min": 0, "max": 3000}
Map.addLayer(image, vis_params, "Sentinel-2 image")

In [25]:
# Create Dynamic World land cover composite
landcover = geemap.dynamic_world(region, start_date, end_date, return_type="hillshade")
Map.addLayer(landcover, {}, "Land Cover")

In [26]:
# Add legend to the map
Map.add_legend(title="Dynamic World Land Cover", builtin_legend="Dynamic_World")
Map

Map(bottom=60882.0, center=[13.480447828815223, 77.19511935799075], controls=(WidgetControl(options=['position…

![](https://i.imgur.com/GEzsSii.png)

In [29]:
# Save Dynamic World class data in GeoTIFF format to Google Drive
output_path = "landcover.tif" # This will be the filename in your Google Drive
landcover = geemap.dynamic_world(region, start_date, end_date, return_type="class")
geemap.ee_export_image_to_drive(
    landcover, description=output_path.split('.')[0], folder="earthengine_exports", scale=10, region=region.geometry()
)